# **Psychology-6k Semantic Retrieve:Il match semantico delle risposte in psicologia**

Il progetto sviluppato realizza un **sistema di retrieval semantico** capace di associare un input (domanda) all’output corretto (risposta) all’interno del dataset Psychology-6k, disponibile su HuggingFace.

Il flusso è organizzato come pipeline:

* **Preparazione e pulizia dei testi** – sono state testate due modalità: una light (normalizzazione di base) e una con lemmatizzazione tramite SpaCy. Quest’ultima non era strettamente necessaria per il progetto, ma è stata integrata a fini didattici.

* **Trasformazione in embeddings** – tramite il modello thenlper/gte-small di SentenceTransformers.

* **Calcolo della similarità coseno** – per stimare la vicinanza semantica tra domande e risposte candidate.

* **Ranking e valutazione** – con la metrica Mean Reciprocal Rank (MRR), che misura la posizione media della risposta corretta nella classifica.

* **Interfaccia interattiva (Gradio)** – per esplorare le domande, confrontare le due pipeline e visualizzare i risultati.

*Risultati*
Il sistema ha ottenuto valori di **MRR molto elevati: ≈0.97** con la pipeline light e ≈0.96 con la pipeline lemma, confermando che la risposta corretta viene classificata quasi sempre al primo posto.

##Caricamento Dataset
Ho caricato il dataset **Psychology-6k** direttamente da HuggingFace (`samhog/psychology-6k`).

**Dettagli del dataset:**
- **Modalities:** Text  
- **Format:** JSON  
- **Size:** 1K – 10K  
- **Librerie utilizzabili:** 🤗 Datasets, 🐼 Pandas

In questa fase verifico la struttura dei dati e preparerò il DataFrame per l’analisi.


In [ ]:
#Caricamento dataset da HuggingFace
from datasets import load_dataset

# Carico il dataset direttamente da HuggingFace
dataset = load_dataset("samhog/psychology-6k")

# Converto la parte 'train' in DataFrame Pandas per analisi
import pandas as pd
df = dataset["train"].to_pandas()

# Mostro le prime righe
df.head()


In [ ]:
#  Analisi Preliminare (EDA )

# 1. Conteggio di record totali
print("Numero di record totali:", len(df))

# 2. Verifico se il dataset ha split (train/valid/test)
print("Split presenti:", dataset.keys())  # es. ['train'] oppure ['train','test']

# 3. Valori mancanti (NaN o stringhe vuote)
print("\nValori mancanti per colonna:")
print(df.isna().sum())
print("\nEsempi di domande vuote:", df[df["input"].isna()])

# 4. Duplicati
dup_questions = df.duplicated(subset=["input"]).sum()
print("\nDomande duplicate:", dup_questions)

dup_q_and_ans = df.duplicated(subset=["input", "output"]).sum()
print("Coppie domanda-risposta duplicate:", dup_q_and_ans)

# 5. Distribuzione lunghezze (in caratteri)
df["q_len_char"] = df["input"].astype(str).str.len()
print("\nStatistiche lunghezze domande (caratteri):")
print(df["q_len_char"].describe())

# Distribuzione lunghezze (in token/parole)
df["q_len_tokens"] = df["input"].astype(str).str.split().apply(len)
print("\nStatistiche lunghezze domande (token):")
print(df["q_len_tokens"].describe())

# Visualizzo un campione di domande molto corte o molto lunghe
print("\nDomande corte:")
print(df[df["q_len_tokens"] < 3]["input"].head())
print("\nDomande lunghe:")
print(df[df["q_len_tokens"] > 30]["input"].head())

### Analisi Esplorativa del Dataset (EDA)

L’analisi esplorativa del dataset *Psychology-6k* ha mostrato quanto segue:

- **Dimensione**: 5846 record, tutti nello split `train` (nessuna divisione train/test predefinita).  
- **Qualità dei dati**: nessun valore mancante o domanda vuota, dati quindi completi.  
- **Duplicati**: circa 1489 domande duplicate (25%). Questo indica che un quarto delle domande si ripete, probabilmente per coprire più formulazioni di casi simili. Tale ridondanza potrebbe influenzare la fase di retrieval, semplificando eccessivamente la ricerca della risposta.  
- **Distribuzione lunghezze**: domande di lunghezza media (≈13 token), con minimo 4 e massimo 43 token. La variabilità è contenuta, con poche domande molto lunghe che descrivono situazioni complesse (es. depressione, ansia, dipendenze).  
- **Struttura**: il dataset non fornisce direttamente una lista di **risposte candidate**. Sono presenti solo la **domanda** (`input`) e la **risposta corretta** (`output`). Per la fase di retrieval sarà quindi necessario costruire un set di risposte candidate, ad esempio campionando da altre risposte del dataset o seguendo le linee guida fornite dal docente.

**Conclusione:**  
Il dataset presenta una buona qualità complessiva: è completo, ben bilanciato nelle lunghezze e senza valori nulli. Gli aspetti critici principali sono:  
1. la presenza di domande duplicate (≈25%), che richiede attenzione in fase di valutazione;  
2. l’assenza di candidate answers predefinite, che dovranno essere generate artificialmente per testare il sistema di retrieval.



## **Pulizia e normalizzazione del testo**


Tecniche di pre-processing utilizzate

In questa fase ho applicato due tipi di pre-processing:

1. **Pulizia leggera**  
   Ho normalizzato il testo portandolo in minuscolo, eliminando caratteri rumorosi (emoji e simboli non alfanumerici) e compattando gli spazi.  
   Questa scelta serve a rendere i dati più uniformi senza alterare troppo il contesto, cosa utile per modelli di tipo SBERT che beneficiano di frasi naturali.

2. **Lemmatizzazione con SpaCy + filtro POS**  
   Ho usato SpaCy per ridurre le parole al loro lemma (es. *running*, *ran* → *run*) e ho mantenuto solo le parole di contenuto (sostantivi, verbi, aggettivi e nomi propri).  
   In questo modo riduco la variabilità morfologica e mi concentro sugli elementi più significativi delle frasi.

Ho quindi applicato **due tecniche linguistiche** (lemmatizzazione e POS tagging).



In [ ]:
# Pulizia e normalizzazione del testo
#importo la libreria

import re
import spacy

# In questa parte definisco due modalità di pulizia:
# - pulizia leggera: normalizza il testo senza stravolgerlo (più adatta per embeddings)
# - lemmatizzazione: riduce le parole alla radice, così riduco la variabilità morfologica


# -----------------------------
# PULIZIA LEGGERA
# -----------------------------
# Qui creo una whitelist di caratteri che voglio mantenere:
# lettere, numeri, spazi e la punteggiatura più comune (? ! . , ecc.)
_NOISE_CHARS = r"[^a-zA-Z0-9\s\.\,\?\!\:\;\-\(\)\'\"]"

def clean_text_light(text: str) -> str:
    """
    Pulizia leggera del testo:
    - converto tutto in minuscolo
    - tolgo caratteri strani (emoji, simboli non utili)
    - compatto gli spazi
    """
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(_NOISE_CHARS, " ", text)   # tolgo caratteri non ammessi
    text = re.sub(r"\s+", " ", text).strip() # elimino spazi multipli
    return text

# Applico la pulizia leggera alle colonne input (domanda) e output (risposta corretta)
df["input_clean"]  = df["input"].apply(clean_text_light)
df["output_clean"] = df["output"].apply(clean_text_light)


# -----------------------------
# LEMMATIZZAZIONE (OPZIONALE)
# -----------------------------
# In questa parte uso SpaCy per portare le parole al loro lemma (radice).
# Esempio: "running", "runs", "ran" diventano "run".
# Lo faccio solo su sostantivi, verbi, aggettivi e nomi propri, perché sono le parole di contenuto.
# Nota: questa fase è più lenta e non sempre migliora gli embeddings contestuali.

# Carico il modello di SpaCy (se non c’è, devo scaricarlo con: !python -m spacy download en_core_web_sm)
try:
    nlp = spacy.load("en_core_web_sm", disable=["parser"])
except OSError:
    nlp = spacy.load("en_core_web_sm", disable=["parser"])

KEEP_POS = {"NOUN", "PROPN", "VERB", "ADJ"}  # i POS che considero utili

def lemmatize_spacy(text: str, remove_stop: bool = True) -> str:
    """
    Lemmatizzazione con filtro POS:
    - mantengo solo le parole di contenuto (sostantivi, verbi, aggettivi, nomi propri)
    - tolgo le stopwords se richiesto
    """
    if not isinstance(text, str):
        return ""
    doc = nlp(text)
    tokens = []
    for tok in doc:
        if remove_stop and tok.is_stop:
            continue
        if tok.pos_ not in KEEP_POS:
            continue
        lemma = tok.lemma_.strip()
        if lemma:
            tokens.append(lemma)
    return " ".join(tokens)

# Creo anche le colonne con la versione lemmatizzata
df["input_lemma"]  = df["input_clean"].apply(lemmatize_spacy)
df["output_lemma"] = df["output_clean"].apply(lemmatize_spacy)


# -----------------------------
# ANTEPRIMA
# -----------------------------
# Qui confronto le tre versioni: testo originale, pulito e lemmatizzato.
df[["input", "input_clean", "input_lemma", "output", "output_clean", "output_lemma"]].head()


### Candidate Answers

Nella fase successiva costruisco i **candidate answers** per ogni domanda.  
Il dataset *Psychology-6k* infatti fornisce solo la domanda (`input`) e la risposta corretta (`output`), ma non una lista di alternative.  

Per poter testare il retrieval, genero quindi per ogni domanda:  
- 1 risposta corretta (gold)  
- 3 risposte errate scelte casualmente dal resto del dataset  

Questa operazione la ripeto **per entrambe le versioni del dataset** (light e lemma), in modo da poter confrontare le prestazioni del sistema in condizioni equivalenti ma con pre-processing diverso.


In [ ]:
#  Costruzione dei Candidate Answers
# In questa fase preparo, per ogni domanda, un set di risposte:
# - 1 risposta corretta (gold)
# - alcune risposte sbagliate prese a caso dal dataset (negatives)
# Questo è necessario perché nel dataset Psychology-6k non ci sono già candidate answers.

import random

# imposto un seme random fisso così i risultati sono riproducibili
random.seed(42)

def build_candidates(df, question_col, answer_col, k_neg=3):
    """
    Per ogni domanda creo un dizionario con:
    - question: la domanda
    - gold: la risposta corretta
    - candidates: lista con 1 gold + k_neg risposte errate prese a caso dal dataset
    """
    all_answers = df[answer_col].tolist()
    N = len(df)
    records = []

    for i, row in df.iterrows():
        q = row[question_col]
        gold = row[answer_col]

        # seleziono k_neg risposte errate diverse dalla gold
        negatives = []
        while len(negatives) < k_neg:
            j = random.randrange(N)
            cand = all_answers[j]
            if cand != gold and cand not in negatives:
                negatives.append(cand)

        candidates = [gold] + negatives
        random.shuffle(candidates)  # mescolo così la gold non è sempre in prima posizione

        records.append({
            "question": q,
            "gold": gold,
            "candidates": candidates
        })
    return records



### Nota: **confronto**

Dopo aver pulito e normalizzato il dataset, ho deciso di preparare **due versioni parallele** dei dati:  

1. **Pulizia leggera (light):** testo convertito in minuscole, rimozione del rumore e gestione spazi, mantenendo la punteggiatura utile.  
2. **Pulizia + Lemmatizzazione (lemma):** stessa pulizia leggera, ma in più riduzione delle parole alla loro radice tramite SpaCy (es. *running*, *ran*, *runs* → *run*).  

L’obiettivo è confrontare questi due approcci e valutare se la lemmatizzazione porta effettivamente un miglioramento nelle prestazioni del retrieval rispetto alla semplice pulizia leggera.


In [ ]:
# Qui preparo i dati per le due modalità:
# - pulizia leggera (light)
# - pulizia + lemmatizzazione (lemma)

# Per la pulizia leggera uso input_clean e output_clean
records_light = build_candidates(df, question_col="input_clean", answer_col="output_clean", k_neg=3)

# Per la pulizia con lemma uso input_lemma e output_lemma
records_lemma = build_candidates(df, question_col="input_lemma", answer_col="output_lemma", k_neg=3)

# Controllo un esempio per ogni modalità
print("Esempio (pulizia leggera):")
print(records_light[0])

print("\nEsempio (pulizia + lemma):")
print(records_lemma[0])

## **Embedding**

### Generazione Embeddings (light vs lemma)

In questa fase trasformo testi (domande e risposte candidate) in vettori numerici usando il modello `thenlper/gte-small`.
Creo due insiemi separati di embeddings:
- **light** → ottenuti dai testi puliti “leggeri”,
- **lemma** → ottenuti dai testi puliti + lemmatizzati.

Scelgo di **deduplicare** i testi prima di codificarli (question + tutti i candidates) per velocizzare e risparmiare memoria.
Normalizzo gli embeddings così che il **prodotto scalare = similarità coseno**, utile per la fase di retrieval.
Ho configurato batch_size di 60

In [ ]:
#  Embeddings con SentenceTransformers (light vs lemma)

# qui importo le librerie necessarie per gli embeddings
# --- Embeddings con SentenceTransformers: versione "pulita" senza cache globale ---

import numpy as np
from sentence_transformers import SentenceTransformer
import pickle

# scelgo il modello con cui lavorare (posso cambiarlo in 1 riga)
MODEL_NAME = "thenlper/gte-small"

def make_model(model_name: str = MODEL_NAME):
    """
    Creo l'istanza del modello una sola volta e la ritorno.
    Preferisco farlo qui (helper) così ho il punto unico dove eventualmente
    configurare device/dtype/log ecc.
    """
    return SentenceTransformer(model_name)

def encode_texts(texts, model: SentenceTransformer, batch_size: int = 60, normalize: bool = True):
    """
    Trasformo una lista di testi in embeddings usando *l'istanza* del modello
    che passo esplicitamente (niente stato globale).
    Se normalize=True, gli embedding sono L2-normalizzati e il dot product = coseno.
    """
    return model.encode(
        texts,
        batch_size=batch_size,
        convert_to_numpy=True,
        show_progress_bar=True,
        normalize_embeddings=normalize
    )

def build_embedding_store(records, model: SentenceTransformer, batch_size: int = 64):
    """
    Costruisco un dizionario {testo -> embedding} per il set di record passato.
    - raccolgo tutte le stringhe (domande + candidati)
    - deduplica per evitare ricalcoli inutili
    - codifico in batch con l'istanza del modello
    """
    uniq_texts = set()
    for rec in records:
        uniq_texts.add(rec["question"])
        for cand in rec["candidates"]:
            uniq_texts.add(cand)

    uniq_texts = list(uniq_texts)
    embs = encode_texts(uniq_texts, model=model, batch_size=batch_size, normalize=True)

    store = {txt: vec for txt, vec in zip(uniq_texts, embs)}
    return store

def store_stats(store, label: str):
    """
    Stampo due numeri per capire 'quanto pesa' lo store:
    - quanti vettori
    - dimensione del vettore
    - stima memoria (float32 ~ 4 byte)
    """
    n_vec = len(store)
    dim = next(iter(store.values())).shape[0] if n_vec > 0 else 0
    approx_mb = n_vec * dim * 4 / (1024**2)
    print(f"{label}: vettori={n_vec}, dim={dim}, ~mem={approx_mb:.1f} MB")

def quick_check(records, store, label: str):
    """
    Controllo veloce: per il primo record verifico che il prodotto scalare
    (coseno se normalizzati) tiri fuori il candidato top.
    """
    print(f"\nQuick check ({label})")
    rec = records[0]
    q = rec["question"]
    cands = rec["candidates"]

    qv = store[q]
    cv = np.stack([store[c] for c in cands], axis=0)

    scores = (cv @ qv).astype(float)
    top_idx = int(np.argmax(scores))

    print("Q    :", q)
    print("GOLD :", rec["gold"])
    print("TOP  :", cands[top_idx])
    print("Scores:", np.round(scores, 3))

# ====== Punto di ingresso: istanzio il modello UNA volta e lo passo dove serve ======

# N.B. assumo che records_light e records_lemma siano già costruiti (fase precedente)
model = make_model(MODEL_NAME)

print("→ Costruisco embedding store (light)…")
emb_store_light = build_embedding_store(records_light, model=model)

print("→ Costruisco embedding store (lemma)…")
emb_store_lemma = build_embedding_store(records_lemma, model=model)

# qualche statistica rapida
store_stats(emb_store_light, "Store LIGHT")
store_stats(emb_store_lemma, "Store LEMMA")

# mini check su un record per modalità
quick_check(records_light, emb_store_light, "light")
quick_check(records_lemma, emb_store_lemma, "lemma")

# salvo su disco per riuso in valutazione (evito ricalcoli)
with open("emb_store_light.pkl", "wb") as f:
    pickle.dump(emb_store_light, f)
with open("emb_store_lemma.pkl", "wb") as f:
    pickle.dump(emb_store_lemma, f)

print("\nEmbeddings pronti per retrieval + MRR (senza cache globale, modello passato esplicitamente).")



### Nota sul log di download e calcolo embeddings

In questa cella Colab prima **scarica il modello `thenlper/gte-small`** da HuggingFace, come si vede dai file elencati (configurazioni, vocabolario, tokenizer e soprattutto `model.safetensors` che contiene i pesi addestrati).  
Questa operazione avviene solo la prima volta: nelle esecuzioni successive il modello viene caricato dalla cache.

Subito dopo inizia il **calcolo degli embeddings**:  
- Ogni frase (domanda + risposte candidate) viene trasformata in un vettore numerico.  
- Il log mostra l’avanzamento a **batch**: ad esempio `64/160` significa 64 batch processati su 160 totali.  
- Sono riportati anche la percentuale di completamento, il tempo già trascorso e il tempo stimato rimanente.

Queste informazioni servono a monitorare che il modello stia lavorando correttamente e a stimare quanto tempo richiede la generazione degli embeddings sull’intero dataset.


##**Retrieval e Valutazione (MRR)**

### Retrieval + Valutazione (MRR)

In questa fase **uso** (cioè *consumo*) gli embeddings calcolati nella precedente processo per:
1) confrontare ogni **domanda** con le sue **candidate answers** via similarità coseno,
2) calcolare la **Mean Reciprocal Rank (MRR)** per misurare quanto spesso la risposta corretta è in alto nel ranking.

Eseguo tutto **due volte**:
- **light** → usando gli embeddings generati dai testi con pulizia leggera,
- **lemma** → usando gli embeddings generati dai testi lemmatizzati.

Confronto i due MRR e mostro anche alcuni esempi “difficili” per analisi qualitativa.


In [ ]:
#  FASE 6 – Retrieval + MRR (confronto light vs lemma)

import numpy as np
import pandas as pd

# qui definisco una funzione che mi calcola la Reciprocal Rank (RR) per una domanda
def reciprocal_rank(gold_idx: int, scores: np.ndarray) -> float:
    """
    RR = 1 / (posizione della risposta corretta nel ranking, 1-based).
    'scores' sono le similarità (coseno) tra domanda e candidati.
    """
    order = np.argsort(-scores)             # indici dei candidati ordinati per score decrescente
    rank_0_based = np.where(order == gold_idx)[0][0]  # posizione 0-based della gold
    return 1.0 / (rank_0_based + 1)         # converto a 1-based

# qui definisco una funzione che valuta un intero set (records + store embeddings)
def evaluate_mrr(records, emb_store, n_examples_to_log=3):
    """
    Scorro tutte le domande:
    - prendo embedding della domanda e dei candidati dallo 'store'
    - calcolo similarità (dot product perché già normalizzati)
    - accumulo le RR per mediare in MRR
    - raccolgo qualche dettaglio qualitativo (casi difficili)
    """
    rr_list = []
    details = []

    for rec in records:
        q = rec["question"]
        cands = rec["candidates"]
        gold = rec["gold"]
        gold_idx = cands.index(gold)

        # recupero i vettori dal magazzino (store) usando il testo come chiave
        qv = emb_store[q]                              # shape: (d,)
        cv = np.stack([emb_store[c] for c in cands])   # shape: (k, d)

        # prodotto scalare = similarità coseno (i vettori sono già normalizzati in Fase 5)
        scores = (cv @ qv).astype(float)               # shape: (k,)

        rr = reciprocal_rank(gold_idx, scores)
        rr_list.append(rr)

        # salvo un po' di info sul caso (mi serve per stampare esempi)
        top_idx = int(np.argmax(scores))
        details.append({
            "question": q,
            "gold": gold,
            "top_pred": cands[top_idx],
            "rr": rr,
            "rank_gold": int(1/rr)
        })

    mrr = float(np.mean(rr_list))
    # ordino i dettagli per mettere in alto i casi peggiori (rank_gold > 1)
    details_sorted = sorted(details, key=lambda d: d["rank_gold"], reverse=True)
    return mrr, details_sorted[:n_examples_to_log]

# qui eseguo la valutazione per entrambe le modalità (light e lemma)
results = {}

print(" Valuto MRR (light)…")
mrr_light, hard_light = evaluate_mrr(records_light, emb_store_light, n_examples_to_log=3)
results["light"] = {"MRR": mrr_light, "hard": hard_light}

print(" Valuto MRR (lemma)…")
mrr_lemma, hard_lemma = evaluate_mrr(records_lemma, emb_store_lemma, n_examples_to_log=3)
results["lemma"] = {"MRR": mrr_lemma, "hard": hard_lemma}

# tabella riassuntiva dei due MRR
summary = pd.DataFrame([
    {"mode": "light", "MRR": results["light"]["MRR"]},
    {"mode": "lemma", "MRR": results["lemma"]["MRR"]}
]).sort_values("MRR", ascending=False)

print("\n Confronto MRR")
display(summary)

# stampo alcuni esempi “difficili” per ciascuna modalità (gold non al rank 1)
def print_hard_examples(mode_label, hard_list):
    print(f"\nEsempi difficili – {mode_label}")
    if not hard_list:
        print("(Nessun esempio difficile nei primi 3 casi richiesti)")
        return
    for ex in hard_list:
        print("-" * 70)
        print("Q   :", ex["question"])
        print("GOLD:", ex["gold"])
        print("TOP :", ex["top_pred"])
        print("RR  :", round(ex["rr"], 3), f"(rank_gold = {ex['rank_gold']})")

print_hard_examples("light", results["light"]["hard"])
print_hard_examples("lemma", results["lemma"]["hard"])


### Conclusioni sul confronto Light vs Lemma

Dopo aver confrontato le due pipeline di pre-processing, i risultati mostrano che:

- **Pulizia leggera (light):** ottiene MRR = 0.975  
- **Pulizia + Lemmatizzazione (lemma):** ottiene MRR = 0.958  

quindi in  entrambi casi il il ranking è quasi sempre al primo posto.
Entrambi i valori sono alti e indicano un sistema di retrieval molto efficace, ma la modalità **light è leggermente superiore**.  
La spiegazione è che i modelli come `gte-small` (SentenceTransformer) sono addestrati su frasi naturali, quindi **beneficiano di mantenere il contesto originale**, inclusa punteggiatura e variazioni morfologiche.  

Con la lemmatizzazione, invece, le frasi diventano più “povere” e astratte (perdono flessioni verbali, tempi e strutture sintattiche), riducendo la ricchezza semantica utile al modello.  

**Conclusione:** in questo dataset conviene usare la **pulizia leggera** per gli embeddings, mentre la versione lemmatizzata rimane utile solo come confronto per mostrare l’impatto del pre-processing.


## GUI Retrieval con Gradio

In [ ]:
#  GUI Retrieval con Gradio (toggle light/lemma, Top-k)
# In questa cella preparo una demo interattiva per mostrare il retrieval:
# - io scelgo se usare pre-processing "light" o "lemma"
# - inserisco una domanda e scelgo Top-k
# - vedo le migliori risposte dal catalogo (tutte le output del dataset)

# 1) importo librerie necessarie
import numpy as np
import pandas as pd

try:
    import gradio as gr
except ImportError:
    # !pip -q install gradio
    import gradio as gr

try:
    from sentence_transformers import SentenceTransformer
except ImportError:
    # !pip -q install sentence-transformers
    from sentence_transformers import SentenceTransformer

# 2) carico una sola volta il modello e lo tengo in cache
_model_cache = {}
def get_model(name: str = "thenlper/gte-small"):
    """Carico l'encoder una sola volta (cache) per non rallentare la GUI."""
    if name not in _model_cache:
        _model_cache[name] = SentenceTransformer(name)
    return _model_cache[name]

# 3) utility: encoding normalizzato (così il dot product = cosine similarity)
def encode_norm(texts, model_name="thenlper/gte-small", batch_size=128):
    """Genero embeddings normalizzati per un elenco di testi."""
    model = get_model(model_name)
    return model.encode(
        texts,
        batch_size=batch_size,
        convert_to_numpy=True,
        show_progress_bar=False,
        normalize_embeddings=True
    )

# 4) preparo i pool di risposte per entrambe le modalità (light/lemma)
#    uso le colonne già create in Fase 3: df['output_clean'] e df['output_lemma']
#    NB: se il dataset contiene duplicati, li rimuovo per velocizzare
answer_pool = {}
answer_pool["light"] = sorted(set(df["output_clean"].dropna().astype(str).tolist()))
answer_pool["lemma"] = sorted(set(df["output_lemma"].dropna().astype(str).tolist()))

# 5) pre-calcolo gli embeddings dei pool (prima volta può richiedere un po’)
#    così a query time calcolo solo l'embedding della domanda (veloce)
pool_embs = {}
for mode in ["light", "lemma"]:
    print(f"↻ Pre-calcolo embeddings del pool risposte [{mode}] …")
    pool_embs[mode] = encode_norm(answer_pool[mode], model_name="thenlper/gte-small")

# 6) funzione di pre-processing della query, coerente con la modalità scelta
def preprocess_query(q: str, mode: str) -> str:
    """Applico lo stesso pre-processing della pipeline al testo della query."""
    if mode == "lemma":
        # prima pulizia leggera, poi lemma (coerente con le mie fasi)
        q_clean = clean_text_light(q)
        q_lemma = lemmatize_spacy(q_clean, remove_stop=True)
        return q_lemma
    else:
        return clean_text_light(q)

# 7) funzione principale usata dalla GUI
def retrieve(query: str, mode: str, topk: int = 3):
    """
    - applico il pre-processing scelto alla query
    - calcolo l'embedding della query
    - calcolo la similarità coseno con tutte le risposte del pool scelto
    - ritorno top-k (risposta, score) + mini nota "perché"
    """
    if not query or not query.strip():
        return [], "Inserisci una domanda per avviare la ricerca."

    # applico il pre-processing coerente
    q_proc = preprocess_query(query, mode)

    # embedding query
    qv = encode_norm([q_proc])[0]  # shape: (d,)

    # prendo il pool e i suoi embeddings pre-calcolati
    cands = answer_pool[mode]
    cv = pool_embs[mode]           # shape: (N, d)

    # similarità coseno = dot product (vettori già normalizzati)
    scores = cv @ qv               # shape: (N,)

    # ordino e prendo i top-k
    topk = int(max(1, min(topk, 10)))
    order = np.argsort(-scores)[:topk]

    results = []
    for idx in order:
        results.append([cands[idx], float(scores[idx])])

    # mini spiegazione: margine tra top-1 e top-2 (se disponibili)
    if len(order) >= 2:
        margin = float(scores[order[0]] - scores[order[1]])
        why = f"Margine top1–top2: {margin:.3f} (più è alto, più la scelta è netta)."
    else:
        why = "Mostro una sola risposta (Top-1)."

    return results, why

# 8) costruisco l'interfaccia Gradio
with gr.Blocks() as demo:
    gr.Markdown("## Retrieval Q→A – Demo (light vs lemma)")
    gr.Markdown(
        "Seleziono il **pre-processing** (light/lemma), inserisco una **domanda** e scelgo **Top-k**. "
        "Vedo le migliori risposte dal pool del dataset. Gli score sono similarità coseno."
    )

    with gr.Row():
        mode = gr.Radio(choices=["light","lemma"], value="light", label="Pre-processing")
        topk = gr.Slider(1, 10, value=3, step=1, label="Top-k")

    query = gr.Textbox(label="Domanda (in inglese)")

    out_tbl = gr.Dataframe(headers=["Risposta", "Score"], datatype=["str","number"], wrap=True)
    out_why = gr.Markdown()

    btn = gr.Button("Cerca")
    btn.click(fn=retrieve, inputs=[query, mode, topk], outputs=[out_tbl, out_why])

# 9) lancio la GUI (condivisibile con link pubblico se share=True)
demo.launch(share=True)


In [ ]:
!jupyter nbconvert --clear-output --ClearMetadataPreprocessor.enabled=True \
  --to notebook --inplace "Psychology-6k Semantic Retrieve:Il match semantico delle risposte in psicologia V2.ipynb"
